In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/mnt/arrakis/ephemeral/vlm-kotlin/server')

In [13]:
from app.jobs import (
    Job,
    get_jobs,
    JobStatus,
    job_queue,
    job_results,
    workers,
    process_jobs,
    start_workers,
    stop_workers,
    periodic_cleanup,
)
from app.deps import get_model, get_tokenizer
from app.logging import setup_logger

from pathlib import Path
from PIL import Image
from rich import print as rich_print
import asyncio
import io


In [3]:
logger = setup_logger()

In [4]:
model = await get_model()
tokenizer = await get_tokenizer()


/mnt/arrakis/ephemeral/vlm-kotlin/server/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/mnt/arrakis/ephemeral/vlm-kotlin/server/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
await start_workers(2, model, tokenizer, logger)

In [6]:
image = Image.open(Path('../data/doctors-note.jpg')).convert('RGB')
image_bytes = io.BytesIO()
image.save(image_bytes, format='JPEG')
image_bytes = image_bytes.getvalue()


job = Job(image=image_bytes)
print(job)

id=UUID('ee3c454b-6e39-405b-a399-902894bf4834') status=<JobStatus.PENDING: 'pending'> image=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x038\x04e\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa

In [7]:
job_results[job.id] = job
await job_queue.put(job.id)

In [23]:
job_results

{}

In [25]:
async def create_and_queue_job(semaphore):
    async with semaphore:
        image = Image.open(Path('../data/doctors-note.jpg')).convert('RGB')
        image_bytes = io.BytesIO()
        image.save(image_bytes, format='JPEG')
        image_bytes = image_bytes.getvalue()

        job = Job(image=image_bytes)
        job_results[job.id] = job
        await job_queue.put(job.id)
        print(f"Created and queued job: {job.id}")


async def main():
    semaphore = asyncio.Semaphore(2)  # Allow 2 concurrent operations
    tasks = [create_and_queue_job(semaphore) for _ in range(5)]  # Create 5 jobs
    await asyncio.gather(*tasks)


# Run the main coroutine
await main()

# Print the number of jobs in the queue
print(f"Number of jobs in the queue: {job_queue.qsize()}")


2024-09-01 20:24:21,325 - vlm-server - INFO - Processing job cedeabe2-a799-4796-8a17-15eea169a0dc
2024-09-01 20:24:23,508 - vlm-server - INFO - Processing job ca4a7bcd-094b-4e69-b9e3-8dcdb2891177
2024-09-01 20:24:33,513 - vlm-server - INFO - Job cedeabe2-a799-4796-8a17-15eea169a0dc completed successfully
2024-09-01 20:24:33,514 - vlm-server - INFO - Processing job e24e3c94-183e-4a09-9be6-8396a1641be6
2024-09-01 20:24:35,699 - vlm-server - INFO - Job ca4a7bcd-094b-4e69-b9e3-8dcdb2891177 completed successfully
2024-09-01 20:24:35,700 - vlm-server - INFO - Processing job bab90e91-ce7e-4fdb-ba5f-af111d34ef40
2024-09-01 20:24:45,588 - vlm-server - INFO - Job e24e3c94-183e-4a09-9be6-8396a1641be6 completed successfully
2024-09-01 20:24:47,770 - vlm-server - INFO - Job bab90e91-ce7e-4fdb-ba5f-af111d34ef40 completed successfully


Created and queued job: cedeabe2-a799-4796-8a17-15eea169a0dc
Created and queued job: ca4a7bcd-094b-4e69-b9e3-8dcdb2891177
Created and queued job: a6bde765-ec29-40f5-b7f5-0084263d6a20
Created and queued job: e24e3c94-183e-4a09-9be6-8396a1641be6
Created and queued job: bab90e91-ce7e-4fdb-ba5f-af111d34ef40
Number of jobs in the queue: 2


In [30]:
get_jobs()

{'total_jobs': 5,
 'pending_jobs': [],
 'processing_jobs': [],
 'completed_jobs': [{'job_id': 'cedeabe2-a799-4796-8a17-15eea169a0dc',
   'status': 'completed',
   'created_at': '2024-09-01T20:24:21.286035',
   'completed_at': '2024-09-01T20:24:33.513865',
   'age': 44.804143,
   'ttr': 12.22783},
  {'job_id': 'ca4a7bcd-094b-4e69-b9e3-8dcdb2891177',
   'status': 'completed',
   'created_at': '2024-09-01T20:24:21.300037',
   'completed_at': '2024-09-01T20:24:35.699312',
   'age': 42.618716,
   'ttr': 14.399275},
  {'job_id': 'a6bde765-ec29-40f5-b7f5-0084263d6a20',
   'status': 'completed',
   'created_at': '2024-09-01T20:24:21.310139',
   'completed_at': '2024-09-01T20:24:37.874920',
   'age': 40.443119,
   'ttr': 16.564781},
  {'job_id': 'e24e3c94-183e-4a09-9be6-8396a1641be6',
   'status': 'completed',
   'created_at': '2024-09-01T20:24:21.317359',
   'completed_at': '2024-09-01T20:24:45.588591',
   'age': 32.729457,
   'ttr': 24.271232},
  {'job_id': 'bab90e91-ce7e-4fdb-ba5f-af111d34ef

In [22]:

await stop_workers()